In [11]:
"""
Script: build_meteo_dataset_retry_cooldown_resume.py
Description:
    Robust extractor for Open-Meteo daily archive with:
      - Sequential per-zone processing
      - Retries with exponential backoff + jitter
      - Global cool-down between zones
      - Per-zone CSV caching/resume (skip re-download if present)

Requirements:
    pip install pandas requests holidays tqdm
"""

import os
import sys
import time
import random
import requests
import pandas as pd
from datetime import datetime
from tqdm import tqdm
import holidays

# ----------------------------
# Logging helper 
# ----------------------------
def log(msg: str) -> None:
    now = datetime.now().strftime("%H:%M:%S")
    print(f"[{now}] {msg}")
    sys.stdout.flush()


# ----------------------------
# CONFIG (adjust as needed)
# ----------------------------
ZONES = {
 
    "LISBOA": (38.7167, -9.1333),  
    "SINTRA": (38.801, -9.3783), 
    "CASCAIS": (38.6968, -9.4215), 
    "LOURES": (38.8309, -9.1684), 
    "AMADORA": (38.7538, -9.2308),
    "OEIRAS": (38.691, -9.3109),
    "ODIVELAS": (38.7927, -9.1838),
    "VILA_FRANCA_DE_XIRA": (38.9552, -8.9897),
    "MAFRA": (38.9379, -9.3276),
    "TORRES_VEDRAS": (39.0911, -9.2586),
    "MONTIJO": (38.7067, -8.9739),
    "SANTARÉM": (39.2333, -8.6833),
    
 
    "PORTO": (41.1496, -8.611),
    "VILA_NOVA_DE_GAIA": (41.124, -8.6124),
    "MATOSINHOS": (41.1821, -8.6891),
    "GONDOMAR": (41.1445, -8.5322), 
    "MAIA": (41.2357, -8.6199),
    "VALONGO": (41.1888, -8.4986),
    "VILA_DO_CONDE": (41.3533, -8.7452),
    "POVOA_DE_VARZIM": (41.3834, -8.7636),
    "PAREDES": (41.2049, -8.3315), 
    "SANTO_TIRSO": (41.3426, -8.4775),
    "PAÇOS_DE_FERREIRA": (41.2766, -8.3762), 
    "PENAFIEL": (41.2084, -8.2828),
    "MIRANDELA": (41.4874, -7.187),
    "MACEDO DE CAVALEIROS": (41.5382, -6.9611),
    
   
    "BRAGA": (41.5503, -8.42),
    "GUIMARÃES": (41.4444, -8.2962),
    "VILA_NOVA_DE_FAMALICAO": (41.408, -8.5198),
    "BARCELOS": (41.5317, -8.6184),
    "VIANA_DO_CASTELO": (41.6932, -8.8329),
    "FELGUEIRAS": (41.3681, -8.194),
    "VILA REAL": (41.3006, -7.7441),
    "BRAGANÇA": (41.8058, -6.7572),
    
   
    "COIMBRA": (40.2056, -8.4195),
    "LEIRIA": (39.7436, -8.8071),
    "VISEU": (40.661, -7.9097),
    "AVEIRO": (40.6443, -8.6455),
    "SANTA_MARIA_DA_FEIRA": (40.9273, -8.5484),
    "FIGUEIRA_DA_FOZ": (40.1508, -8.8618),
    "OVAR": (40.8586, -8.6251),
    "ALCOBAÇA": (39.5522, -8.9775),
    "OLIVEIRA_DE_AZEMÉIS": (40.841, -8.4756),
    
   
    "SETÚBAL": (38.5244, -8.8882),
    "ALMADA": (38.679, -9.1569),
    "SEIXAL": (38.6401, -9.1014), 
    "BARREIRO": (38.6631, -9.0724),
    "PALMELA": (38.569, -8.9013),
    "MOITA": (38.6508, -8.9904),
    "SESIMBRA": (38.4445, -9.1015), 
    
    # Alentejo e Algarve
    "ÉVORA": (38.5667, -7.9),
    "FARO": (37.0187, -7.9272),
    "LOULÉ": (37.1377, -8.0197),
    "PORTIMÃO": (37.1369, -8.5378),
}

START_DATE = "2015-01-01"
END_DATE   = "2025-10-05"
TIMEZONE   = "Europe/Lisbon"

DAILY_VARS = [
    "temperature_2m_max",
    "temperature_2m_min",
    "temperature_2m_mean",
    "precipitation_sum",
    "wind_speed_10m_max",
    "wind_gusts_10m_max",
    "shortwave_radiation_sum",
    "sunshine_duration",
    "relative_humidity_2m_mean",
    "cloudcover_mean",
    "sunrise",
    "sunset",
]

RENAME_COLS = {
    "temperature_2m_max": "tmax_c",
    "temperature_2m_min": "tmin_c",
    "temperature_2m_mean": "tmean_c",
    "precipitation_sum": "precip_mm",
    "wind_speed_10m_max": "wind_speed_max",
    "wind_gusts_10m_max": "wind_gusts_max",
    "shortwave_radiation_sum": "rad_solar",
    "sunshine_duration": "sunshine_sec",
    "relative_humidity_2m_mean": "humidade_relativa",
    "cloudcover_mean": "nebulosidade_media",
}

# Retry / cooldown policy
MAX_RETRIES_PER_ZONE = 10       # up to 10 attempts per zone
BASE_WAIT_SECONDS    = 10       # exponential backoff base
LONG_COOLDOWN_AFTER  = 5        # after this attempt, apply long cooldown
LONG_COOLDOWN_SECONDS = 180     # 3 minutes cool-down if still rate-limited
COOLDOWN_BETWEEN_ZONES = 30     # 30s after each successful zone
STRICT_ABORT = True             # abort if a zone cannot be fetched after all retries

OUTPUT_CSV = "dataset_meteo_zonal.csv"
RAW_DIR = "raw_zones"           # per-zone cache directory


# ----------------------------
# Helpers
# ----------------------------
def ensure_dir(path: str):
    if not os.path.isdir(path):
        os.makedirs(path, exist_ok=True)


def save_zone_csv(df: pd.DataFrame, zone: str):
    ensure_dir(RAW_DIR)
    out_path = os.path.join(RAW_DIR, f"raw_{zone}.csv")
    df.to_csv(out_path, index=False, float_format="%.3f")
    log(f"Saved cache: {out_path}")


def load_zone_csv(zone: str) -> pd.DataFrame:
    path = os.path.join(RAW_DIR, f"raw_{zone}.csv")
    if os.path.isfile(path):
        try:
            df = pd.read_csv(path, parse_dates=["date"])
            log(f"Loaded cached zone: {path} ({len(df)} rows)")
            return df
        except Exception as e:
            log(f"⚠ Failed to load cache for {zone}: {e}")
    return pd.DataFrame()


# ----------------------------
# Single-zone request
# ----------------------------
def fetch_openmeteo_daily(lat: float, lon: float, zone: str, timeout: int = 60):
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": START_DATE,
        "end_date": END_DATE,
        "daily": ",".join(DAILY_VARS),
        "timezone": TIMEZONE,
    }
    log(f"→ Requesting {zone} ({lat}, {lon}) ...")
    retry_after = 0
    try:
        r = requests.get(url, params=params, timeout=timeout)
        retry_after = int(r.headers.get("Retry-After", "0") or "0")
    except Exception as e:
        log(f"✖ Network/connection error for {zone}: {e}")
        return pd.DataFrame(), None, 0

    if r.status_code != 200:
        log(f"✖ HTTP {r.status_code} for {zone}")
        return pd.DataFrame(), r.status_code, retry_after

    data = r.json()
    if "daily" not in data:
        log(f"⚠ No 'daily' key in response for {zone}")
        return pd.DataFrame(), 200, 0

    df = pd.DataFrame(data["daily"])
    if "time" not in df.columns:
        log(f"⚠ Missing 'time' column for {zone}")
        return pd.DataFrame(), 200, 0

    df["date"] = pd.to_datetime(df["time"])
    df["zone"] = zone
    df["latitude"] = lat
    df["longitude"] = lon
    log(f"✓ {zone}: {len(df)} rows")
    return df, 200, 0


# ----------------------------
# Retry wrapper (sequential)
# ----------------------------
def fetch_zone_with_retry(zone: str, lat: float, lon: float) -> pd.DataFrame:
    # Try cache first
    cached = load_zone_csv(zone)
    if not cached.empty and cached["date"].notna().any():
        return cached

    for attempt in range(1, MAX_RETRIES_PER_ZONE + 1):
        df, status, retry_after = fetch_openmeteo_daily(lat, lon, zone, timeout=60)

        if status == 200 and not df.empty:
            save_zone_csv(df, zone)
            return df

        # decide waiting time
        if status == 429 and retry_after > 0:
            wait = retry_after + 1
            log(f"↻ Rate limited for {zone}. Respecting Retry-After: waiting {wait}s before retry #{attempt}.")
        else:
            wait = BASE_WAIT_SECONDS * (2 ** (attempt - 1))
            wait = int(wait + random.uniform(0, 1) * BASE_WAIT_SECONDS * 0.5)
            log(f"↻ Retry {attempt}/{MAX_RETRIES_PER_ZONE} for {zone} in {wait}s ...")

        # long cool-down after several failed attempts
        if attempt == LONG_COOLDOWN_AFTER:
            log(f"⏳ Applying long cool-down of {LONG_COOLDOWN_SECONDS}s due to repeated rate limits/errors for {zone}.")
            time.sleep(LONG_COOLDOWN_SECONDS)

        time.sleep(wait)

    msg = f"✖ Exhausted retries for {zone}."
    if STRICT_ABORT:
        log(msg + " Aborting to avoid incomplete dataset.")
        sys.exit(1)
    else:
        log(msg + " Skipping this zone and continuing.")
        return pd.DataFrame()


# ----------------------------
# STEP 1: Download (sequential, with resume and cool-down)
# ----------------------------
log("=== STEP 1: Downloading zones (sequential with retries, cache & cool-down) ===")
tables = []
for zone, (lat, lon) in ZONES.items():
    log(f"--- Starting zone: {zone} ---")
    df_zone = fetch_zone_with_retry(zone, lat, lon)
    tables.append(df_zone)
    log(f"--- Finished zone: {zone} ---")
    log(f"Cooling down {COOLDOWN_BETWEEN_ZONES}s before next zone ...")
    time.sleep(COOLDOWN_BETWEEN_ZONES)

df = pd.concat(tables, ignore_index=True)
df = df.sort_values(["zone", "date"]).reset_index(drop=True)
log(f"✓ Combined shape: {df.shape[0]:,} rows × {df.shape[1]} cols")


# ----------------------------
# STEP 2: Clean & rename
# ----------------------------
log("=== STEP 2: Cleaning and renaming columns ===")
df = df.rename(columns=RENAME_COLS)
for col in RENAME_COLS.values():
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")
log("✓ Columns standardized and converted to numeric.")


# ----------------------------
# STEP 3: Derived variables
# ----------------------------
log("=== STEP 3: Creating derived variables (HDD, CDD, amplitude, day length) ===")
if "tmean_c" in df.columns:
    df["HDD18"] = (18 - df["tmean_c"]).clip(lower=0)
    df["CDD22"] = (df["tmean_c"] - 22).clip(lower=0)
else:
    df["HDD18"] = pd.NA
    df["CDD22"] = pd.NA

if {"tmax_c", "tmin_c"}.issubset(df.columns):
    df["amp_termica"] = df["tmax_c"] - df["tmin_c"]
else:
    df["amp_termica"] = pd.NA

if {"sunrise", "sunset"}.issubset(df.columns):
    df["sunrise"] = pd.to_datetime(df["sunrise"], errors="coerce")
    df["sunset"]  = pd.to_datetime(df["sunset"],  errors="coerce")
    df["day_length_hours"] = ((df["sunset"] - df["sunrise"]).dt.total_seconds() / 3600).round(2)
else:
    df["day_length_hours"] = pd.NA

log("✓ Derived variables created.")


# ----------------------------
# STEP 4: Calendar features
# ----------------------------
log("=== STEP 4: Adding calendar features (DOW, weekend, month, year, holidays, DST) ===")
df["dow"] = df["date"].dt.dayofweek
df["is_weekend"] = df["dow"].isin([5, 6]).astype(int)
df["month"] = df["date"].dt.month
df["year"] = df["date"].dt.year

try:
    pt_holidays = holidays.Portugal()
    df["is_holiday"] = df["date"].isin(pt_holidays).astype(int)
except Exception as e:
    log(f"⚠ Holidays feature failed: {e}")
    df["is_holiday"] = 0

try:
    df["is_dst"] = df["date"].apply(lambda d: bool(pd.Timestamp(d, tz="Europe/Lisbon").dst()))
except Exception as e:
    log(f"⚠ DST feature failed: {e}")
    df["is_dst"] = 0

log("✓ Calendar features added.")


# ----------------------------
# STEP 5: Save
# ----------------------------
log("=== STEP 5: Saving final dataset ===")
df.to_csv(OUTPUT_CSV, index=False, float_format="%.3f")
log(f"Saved to: {OUTPUT_CSV}")
log(f"Rows: {df.shape[0]:,} | Columns: {df.shape[1]}")
log("Done. Ready for feature analysis and regression!")

[01:47:56] === STEP 1: Downloading zones (sequential with retries, cache & cool-down) ===
[01:47:56] --- Starting zone: LISBOA ---
[01:47:56] Loaded cached zone: raw_zones\raw_LISBOA.csv (3931 rows)
[01:47:56] --- Finished zone: LISBOA ---
[01:47:56] Cooling down 30s before next zone ...
[01:48:26] --- Starting zone: SINTRA ---
[01:48:26] Loaded cached zone: raw_zones\raw_SINTRA.csv (3931 rows)
[01:48:26] --- Finished zone: SINTRA ---
[01:48:26] Cooling down 30s before next zone ...
[01:48:56] --- Starting zone: CASCAIS ---
[01:48:56] Loaded cached zone: raw_zones\raw_CASCAIS.csv (3931 rows)
[01:48:56] --- Finished zone: CASCAIS ---
[01:48:56] Cooling down 30s before next zone ...
[01:49:26] --- Starting zone: LOURES ---
[01:49:26] Loaded cached zone: raw_zones\raw_LOURES.csv (3931 rows)
[01:49:26] --- Finished zone: LOURES ---
[01:49:26] Cooling down 30s before next zone ...
[01:49:56] --- Starting zone: AMADORA ---
[01:49:56] Loaded cached zone: raw_zones\raw_AMADORA.csv (3931 rows)


In [12]:
import pandas as pd
import os



print("Starting the final merge...")


files_needed = ['dataset_meteo_zonal.csv', 'target_consumo.csv']
for f in files_needed:
    if not os.path.exists(f):
        print(f"ERROR: File '{f}' not found. Ensure that the consumption file is in the folder and is named 'target_consumo.csv'.")
        exit()

try:
    
    df_meteo = pd.read_csv('dataset_meteo_zonal.csv')
   
    df_consumo = pd.read_csv('target_consumo.csv') 
except Exception as e:
    print(f"ERROR loading files: {e}")
    exit()


df_meteo['date'] = pd.to_datetime(df_meteo['date'])
df_consumo['date'] = pd.to_datetime(df_consumo['date']) 


df_consumo.rename(columns={'date': 'date'}, inplace=True)


df_final = pd.merge(
    df_meteo,
    df_consumo,
    on='date',
    how='left'
)


df_final.sort_values(by=['date', 'zone'], inplace=True)
df_final.to_csv('dataset_meteo_com_consumo.csv', index=False, float_format="%.3f")

print("\n" + "-" * 50)
print("✅ SUCCESS! DATABASE COMPLETED!")
print(f"The final delivery file ‘dataset_meteo_com_consumo.csv’ was created with {len(df_final):,} lines and {df_final.shape[1]} columns.")
print("-" * 50)

A iniciar o merge final...

--------------------------------------------------
✅ SUCESS! FINALIZED DATABASE!
The final delivery file ‘dataset_meteo_com_consumo.csv’ was created with 212274 rows and 28 columns.
--------------------------------------------------
